This notebook shows how to do PSI with openmined_psi. Nothing to do with duet or syft here.

In [1]:
import openmined_psi as psi

In [2]:
psi.__version__

'0.3.4'

# configuration

Here are the datasets that the client and server keep.

In [3]:
server_items = ['element_{}'.format(i) for i in range(100)]
client_items = ['element_{}'.format(i) for i in range(70,170)]

So we expect an intersection of size 30.

Let's go through the PSI process step by step.

# (1) Server send reveal_intersection

Server chooses whether reveal intersection or not.

In [4]:
reveal_intersection = False

Then the server should send `reveal_intersection` to client. As it's of type bool, we can send it with duet.

# (2) Server create psi.server object

In [5]:
server = psi.server.CreateWithNewKey(reveal_intersection)

# (3) Client create psi.client object

In [6]:
client = psi.client.CreateWithNewKey(reveal_intersection)

# (4) Server send false-positive rate

Server choose a false positive rate.

In [7]:
fpr = 1e-4

The client does not need fpr to do PSI, but maybe the client wants to know it. So the server can choose to send or not send.

If the server choose to send it, we can send it with duet, as it's a float.

# (5) Client send size of set

The server needs the size of client's dataset to create setup message.

In [8]:
client_ds_size = len(client_items)

Then the client should send `client_ds_size` to server. This can be done using duet, as it's an int.

# (6) Server send setup message

Server creates a setup message.

In [9]:
server_setupMsg = server.CreateSetupMessage(fpr, client_ds_size, server_items)
type(server_setupMsg)

private_set_intersection.proto.psi_pb2.ServerSetup

Then the server should send it to client.

When server sends `server_setupMsg`, what is sent is the bytes generated by the protobuf compilier. We can get this bytes using the `.SeerializeToString` method.

In [10]:
server_setupMsg_bytes = server_setupMsg.SerializeToString()
type(server_setupMsg_bytes)

bytes

When the client has received `server_setupMsg_bytes`, the initial setup message object can be recreated the following way:

In [11]:
client_recreate_setupMsg = psi.ServerSetup()
client_recreate_setupMsg.ParseFromString(server_setupMsg_bytes)
client_recreate_setupMsg == server_setupMsg

True

And we want to do this send and receive process with duet, that's a TODO thing.

# (7) Client send request

The client creates a request.

In [12]:
client_request = client.CreateRequest(client_items)
type(client_request)

private_set_intersection.proto.psi_pb2.Request

Then the client should send it to the server. The way is same as server sending setup message.

In [13]:
client_request_bytes = client_request.SerializeToString()
type(client_request_bytes)

bytes

When the server has received `client_request_bytes`, the initial request object can be recreated the same way as client recreating setup message.

In [14]:
server_recreate_request = psi.Request()
server_recreate_request.ParseFromString(client_request_bytes)
client_request == server_recreate_request

True

# (8) Server send response

The server create a response.

In [15]:
server_response = server.ProcessRequest(server_recreate_request)
type(server_response)

private_set_intersection.proto.psi_pb2.Response

Then the server should send it to the client. The way is same server sending setup message.

In [16]:
server_response_bytes = server_response.SerializeToString()
type(server_response_bytes)

bytes

When the client has received `server_response_bytes`, the initial response object can be recreated the same way as client recreating setup message.

In [17]:
client_recreate_response = psi.Response()
client_recreate_response.ParseFromString(server_response_bytes)
client_recreate_response == server_response

True

# (9) Client get the intersection

After the client has received the setup_message and the response, the intersection or the size of intersection can be found out, deponding on reveal_intersection.

In [18]:
if reveal_intersection:
    res = client.GetIntersection(client_recreate_setupMsg, client_recreate_response)
else:
    res = client.GetIntersectionSize(client_recreate_setupMsg, client_recreate_response)

Let's see if we have done the PSI right.

In [19]:
res

30